# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. 

Цель проекта: на основании статистики о платежеспособности клиентов выявить закономерности для создания модели *кредитного скоринга* — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку. 
Основные критерии, влияние которых нужно рассмотреть: семейное положение, наличе детей, уровень дохода и цель кредита.

## Шаг 1. Изучаем общую информацию

Импортируем библиотеки *pandas* и *numpy*.
Прочитаем файл со статистическими данными и выведем часть строк для ознакомления с содержимым.

In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv('/datasets/data.csv')
display(data.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


С помощью метода *.info()* получим общую информацию о данных загруженной таблицы.

In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

В таблице 12 столбцов разного типа:

*children* — количество детей в семье

*days_employed* — общий трудовой стаж в днях

*dob_years* — возраст клиента в годах

*education* — уровень образования клиента

*education_id* — идентификатор уровня образования

*family_status* — семейное положение

*family_status_id* — идентификатор семейного положения

*gender* — пол клиента

*income_type* — тип занятости

*debt* — имел ли задолженность по возврату кредитов

*total_income* — ежемесячный доход

*purpose* — цель получения кредита

В двух столбцах присутствуют пропущенные значения. С заголовками проблем нет. Форматы корректные, можно лишь привести к целочисленному типу данные в столбцах с общим трудовым стажем и ежемесячным доходом.

## Шаг 2. Предобработка данных

Исключим пропуски, проверим данные на наличие дубликатов, а также разобъем данные на категории для более удобного анализа.

### Обработка пропусков

В предыдущем шаге мы выявили, что в данных присутствуют пропущенные значения. Проверим наличие пропусков вызовом набора методов для суммирования пропущенных значений *isnull()* и *sum()*.

In [3]:
data.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Информация о наличии пропусков в двух столбцах подтвердилась. 

Выведем часть таблицы с ячейками с пропусками, воспользовавшись комбинацией методов isna() и sample().

In [4]:
display (data[data['total_income'].isna()].sample(10)) 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2942,0,NaN,66,среднее,1,гражданский брак,1,F,пенсионер,0,NaN,свадьба
7025,0,NaN,32,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,высшее образование
8583,0,NaN,58,высшее,0,Не женат / не замужем,4,F,пенсионер,0,NaN,дополнительное образование
15214,3,NaN,36,среднее,1,гражданский брак,1,F,сотрудник,1,NaN,сделка с автомобилем
2081,1,NaN,42,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,покупка жилья
3092,2,NaN,34,СРЕДНЕЕ,1,в разводе,3,F,сотрудник,0,NaN,свой автомобиль
1092,0,NaN,49,среднее,1,Не женат / не замужем,4,F,сотрудник,0,NaN,покупка жилья для сдачи
5802,0,NaN,29,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,операции с коммерческой недвижимостью
3492,0,NaN,30,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,покупка жилья для семьи
17218,2,NaN,30,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство недвижимости


Пропуски присутствуют в двух столбцах: общий трудовой стаж в днях и ежемесячный доход. Т.к. в обоих случаях количество пропусков одинаковое, и исходя из фрагмента выше, вероятно нет данных по одним и тем же заёмщикам. Возможно данные не были предоставлены или возникли ошибки при обработке и внесении информации в базу данных.

В остальном какой-либо закономерности с пропусками не наблюдается. Строки с пропусками составляют примерно 10% от всей выборки, что существенно. 
Столбец с трудовым стажем для решения поставленной задачи не важен, заполним его нулями.
 

In [5]:
data['days_employed'] = data['days_employed'].fillna(0)

Пропуски в данных по ежемесячному доходу желательно заполнить. Получим список уникальных значений типов занятости и используем медианные значения в зависимости от типа занятости для заполнения пропусков.



In [6]:
data['total_income'].fillna(value = data.groupby(['income_type'])['total_income'].transform('median'), inplace = True)

Проверим результат - количество оставшихся пропусков:

In [7]:
print (data.isnull().sum()) 

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


Пропуски после замены отсутствуют.

Проверим значения в столбце "Семейное положение"

In [8]:
print(data['family_status'].value_counts())
print(data['family_status'].count())

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64
21525


Проблем со столбцом "Семейное положение" не наблюдается. Задвоенных или некорректных значений нет. 

Проанализируем содержание столбца "Количество детей"

In [9]:
print(data['children'].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


Есть отрицательные значения "-1". Вероятно, это связано с неточностями при заполнении.  Исправим это при помощи метода *.abs*,т.е. присвоив всем данным в столбце *children* их значения по модулю, исходя из того, что вместо "-1" должно быть "1".

In [10]:
data['children'] = data['children'].abs()

Так же есть 76 случаев, когда количество детей равно 20. Здесь явно ошибка, т.к. в промежутке от 5 до 20 нет ни одного варианта, а здесь сразу 76. Наиболее вероятно, что в этих случаях детей двое. Но нужно донести эту информацию до тех, кто занимается сбором и заполнением данных. Возможно, не корректно сделана форма для сбора информации. Как вариант, в форме изначально указан ноль, и при внесении значения ноль не исчезает, а к нему добавляется внесенное значение.

Заменим все значения "20" на "2", воспользовавшись *.replace()*.

In [11]:
data['children'] = data['children'].replace(20, 2)
print(data['children'].value_counts())

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64


Теперь ошибок в столбце с количеством детей нет.

**Вывод**

Провели работу с пропусками: пропуски в ежемесячном доходе заполнили медианным значением по типу дохода, пропуски в трудовом стаже заменили на ноль, чтобы можно было в дальнейшем поменять тип данных в столбце на целочисленный.

### Замена типа данных

Приведем данные в столбцах с трудовым стажем и общим доходом к целочисленному типу. Используем метод *.astype()*, который позволяет преобразовать практически любой тип данных в практически любой другой, а также позволяет четко указать, какой тип данным мы хотим получить на выходе. Передадим ему в качестве аргумента **'int'**.
Для проверки результата вызовем метод *info()*.

In [12]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

Типы данных в столбцах удобны для дальнейшей работы.

### Обработка дубликатов

Установим наличие дубликатов и удалим их, чтобы не исказить результаты дальнейшего анализа.

Используем метод *duplicates()* для обнаружения дубликатов.

In [13]:
data[data.duplicated()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,0,41,среднее,1,женат / замужем,0,F,сотрудник,0,142594,покупка жилья для семьи
4182,1,0,34,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,142594,свадьба
4851,0,0,60,среднее,1,гражданский брак,1,F,пенсионер,0,118514,свадьба
5557,0,0,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу
7808,0,0,57,среднее,1,гражданский брак,1,F,пенсионер,0,118514,на проведение свадьбы
8583,0,0,58,высшее,0,Не женат / не замужем,4,F,пенсионер,0,118514,дополнительное образование
9238,2,0,34,среднее,1,женат / замужем,0,F,сотрудник,0,142594,покупка жилья для сдачи
9528,0,0,66,среднее,1,вдовец / вдова,2,F,пенсионер,0,118514,операции со своей недвижимостью
9627,0,0,56,среднее,1,женат / замужем,0,F,пенсионер,0,118514,операции со своей недвижимостью
10462,0,0,62,среднее,1,женат / замужем,0,F,пенсионер,0,118514,покупка коммерческой недвижимости


Посчитаем количество дубликатов, добавив в комбинации к предыдущему метод *.sum()*:

In [14]:
data.duplicated().sum()

54

В данных присутствуют 54 повторяющиеся строки.

Также заметно, что в столбце с образованием присутствуют одинаковые варианты с разным регистром. Просмотрим все использованные в этом столбце варианты, применив *.value_counts()* к столбцу Образование.

In [15]:
print(data['education'].value_counts())

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64


Скорректируем написание в столбце "Образование" в одном регистре. И заново проверим присутствующие в столбце варианты.

In [16]:
data['education'] = data['education'].str.lower()

print(data['education'].value_counts())

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64


Теперь гораздо лучше, варианты не повторяются.

Проверим еще раз количество дубликатов, с учетом операции выше.

In [17]:
data.duplicated().sum()

71

Видим, что корректировка регистра помогла выявить еще несколько дубликатов.

Удаляем дубликаты и дополнительно вызываем метод *reset_index()* для перезаписи индексов в порядке возрастания. Проверяем результат:

In [18]:
data = data.drop_duplicates().reset_index(drop=True)
data.duplicated().sum()


0

Дубликаты отсутствуют.

**Вывод**

Часть дубликатов возникла из-за заполнения столбца "Образование" в разном регистре. Можно этого избежать, если составить ограниченный список для выбора при заполнении или переносе в электронную форму. Стоит разобраться с причинами появления остальных дубликатов.

### Лемматизация

Столбец "Цель кредита" заполняется не типовыми значениями, в нем указаны произвольные варианты. Посчитаем количество возможных вариантов.

In [19]:
display(len(data['purpose'].unique()))

38

38 вариантов для анализа совсем не годится.
Выведем список всех возможных значений, чтобы оценить разнообразие и определить дальнейшие действия по обработке

In [20]:
display(data['purpose'].unique())

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об


Информация в таком виде не удобна для анализа. Обработаем их для дальнейшей категоризации.
Воспользуемся лемматизацией, т.е. процессом приведение слова к его словарной форме (лемме).
Для этого выделим леммы и выведем их в отдельный столбец **'lemmas'**.

In [21]:
from pymystem3 import Mystem #импортируем Mystem из библиотеки pymystem3 для анализа текста
m = Mystem() 

data['lemmas'] = data['purpose'].apply(m.lemmatize)
display(data.sample(5))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas
18217,0,-813,29,среднее,1,гражданский брак,1,M,сотрудник,0,111228,на проведение свадьбы,"[на, , проведение, , свадьба, \n]"
12468,2,-1324,32,высшее,0,женат / замужем,0,M,сотрудник,0,341733,заняться образованием,"[заниматься, , образование, \n]"
12235,0,-2793,38,среднее,1,гражданский брак,1,M,сотрудник,0,161758,свадьба,"[свадьба, \n]"
14087,0,355656,59,среднее,1,Не женат / не замужем,4,F,пенсионер,0,86911,автомобили,"[автомобиль, \n]"
3208,0,-3212,44,среднее,1,гражданский брак,1,F,сотрудник,0,116088,свадьба,"[свадьба, \n]"


**Вывод**

Провели лемматизацию данных в столбце "Цель кредита". Это поможет выделить наиболее популярные цели и произвести группировку данных по этому критерию.

### Категоризация данных

Для решения поставленной задачи, нам нужно проанализировать данные по семейному положению, количеству детей, доходу и цели кредита в контексте наличия просрочек. Часть данных уже разбита на удобные для анализа категории, в частности, семейное положение имеет 5 возможных вариантов. Можно сделать еще более крупные группы: например, объединить "женат / замужем" и "гражданский брак" в одну группу, а "вдовец / вдова" и "в разводе" в другую. Но в данном случае получим слишком крупную группировку, что может исказить результаты. Пять категорий вполне удобны для анализа. 

Для категоризации данных по количеству детей, доходу и цели кредита необходима дополнительная обработка.

**Группировка в зависимости от цели кредита**

На основании столбца **'lemmas'** с полученными леммами выделим основные цели получения кредита.
Посчитаем наиболее часто встречающиеся в цели слова.

In [22]:
from collections import Counter #вызываем специальный контейнер Counter из модуля collections для подсчёта числа упоминаний слов в тексте
print(Counter(data['lemmas'].sum()))


Counter({' ': 33570, '\n': 21454, 'недвижимость': 6351, 'покупка': 5897, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2918, 'операция': 2604, 'свадьба': 2324, 'свой': 2230, 'на': 2222, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'проведение': 768, 'сыграть': 765, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'подержанный': 486, 'подержать': 478, 'приобретение': 461, 'профильный': 436})


Среди часто встречающихся вариантов слова "покупка", "операция". Они могут относится к разным целям, поэтому их игнорируем. Обратим внимание на более очевидные варианты.

Слово "недвижимость" может встречаться в вариантах как для коммерческого использования, так и для жилья. Поэтому объединим жилье и недвижимость в одну категорию.
Получаем основные цели кредита: жилье/недвижимость, автомобиль, образование, свадьба. Если есть цели, которые не попадают ни в одну из указанных категорий, объединим их в категорию "Прочее".
Объявим  функцию для отнесения заемщика к одной из выделенных выше категорий и вызовем функцию для группировки наших данных.

In [23]:
def purpose_lemmas(purpose):
    if 'недвижимость' in purpose or 'жилье' in purpose:
        return 'недвижимость'
    if 'автомобиль' in purpose:
        return 'автомобиль'
    if 'образование' in purpose:
        return 'образование'
    if 'свадьба' in purpose:
        return 'свадьба'
    return 'Прочее'
         
        
data['purpose_main'] = data['lemmas'].apply(purpose_lemmas)
        

Выделили основной признак цели займа в отдельном столбце **'purpose_main'**.
Посчитаем количество вариантов по выделенным группам. Также выведем общее количество элементов в столбце.

In [24]:
print(data['purpose_main'].value_counts())
print(data['purpose_main'].count())

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_main, dtype: int64
21454


Все данные попали одну из четырех указанных категорий. Категория "прочее" не пригодилась.

**Группировка в зависимости от дохода**

Разобьем заёмщиков на группы в зависимости от дохода.
Воспользуемся методом *describe()*, чтобы определить границы групп.

In [25]:
data['total_income'].describe().round(2)

count      21454.00
mean      165319.57
std        98187.30
min        20667.00
25%       107623.00
50%       142594.00
75%       195820.25
max      2265604.00
Name: total_income, dtype: float64

Метод *describe()* среди прочего дает возможность получить такую информацию, как процентиль.
Чтобы группы получились примерно одинаковые по численности, установим следующие диапазоны доходов:

**низкий**: от 0 до 107623р,

**ниже среднего**: от 107624р до 142594р,

**выше среднего**: от 142595р до 195820р

**высокий**: более 195 821р

В новом столбце **total_income_cut** укажем к какой из групп относится заемщик.

In [26]:
data['total_income_cut'] = pd.cut(data['total_income'],[0, data['total_income'].describe()['25%'], data['total_income'].describe()['50%'], data['total_income'].describe()['75%'], np.inf], 
                                  labels=["низкий (до 108т.р.)", "ниже среднего (108-143т.р.)", "выше среднего (143-196т.р.)", "высокий (более 196т.р.)"])
data.sample(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas,purpose_main,total_income_cut
14688,1,-1053,43,среднее,1,женат / замужем,0,F,сотрудник,0,226527,покупка своего жилья,"[покупка, , свой, , жилье, \n]",недвижимость,высокий (более 196т.р.)
5027,0,-661,27,среднее,1,женат / замужем,0,F,госслужащий,0,45934,сделка с автомобилем,"[сделка, , с, , автомобиль, \n]",автомобиль,низкий (до 108т.р.)
7319,0,-3829,46,высшее,0,женат / замужем,0,M,сотрудник,0,221489,покупка недвижимости,"[покупка, , недвижимость, \n]",недвижимость,высокий (более 196т.р.)
15407,0,-1107,54,среднее,1,вдовец / вдова,2,F,сотрудник,0,145871,недвижимость,"[недвижимость, \n]",недвижимость,выше среднего (143-196т.р.)
3417,0,-2026,58,среднее,1,Не женат / не замужем,4,F,госслужащий,0,52405,строительство собственной недвижимости,"[строительство, , собственный, , недвижимост...",недвижимость,низкий (до 108т.р.)


Получили новый столбец **total_income_cut** с делением на категории в зависимости от уровня дохода 



**Группировка в зависимости от количества детей**

Посмотрим варианты в столбце "Количество детей" и оценим потребность в корректировке или группировке.

In [27]:
print(data['children'].value_counts())

0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64


Заёмщики с тремя и более детьми составляют небольшую долю в выборке. Объединим их в группу **"Три и более"**. Другие варианты оставим без группировки. Добавим названия для всех новых групп: **"Нет детей", "Один ребенок", "Два ребенка"**

In [28]:
data['children_grouped'] = pd.cut(data['children'],[-0.1, 0.9, 1.9, 2.9, np.inf], labels=["Нет детей", "Один ребенок", "Два ребенка", "Три и более"])
data.sample(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas,purpose_main,total_income_cut,children_grouped
20552,1,-1709,31,среднее,1,женат / замужем,0,M,компаньон,0,152367,строительство собственной недвижимости,"[строительство, , собственный, , недвижимост...",недвижимость,выше среднего (143-196т.р.),Один ребенок
6704,0,374483,55,среднее,1,женат / замужем,0,F,пенсионер,0,232889,покупка жилой недвижимости,"[покупка, , жилой, , недвижимость, \n]",недвижимость,высокий (более 196т.р.),Нет детей
4539,1,-1811,32,среднее,1,женат / замужем,0,F,сотрудник,0,160544,образование,"[образование, \n]",образование,выше среднего (143-196т.р.),Один ребенок
15849,0,-610,26,среднее,1,Не женат / не замужем,4,F,сотрудник,0,115564,покупка жилья,"[покупка, , жилье, \n]",недвижимость,ниже среднего (108-143т.р.),Нет детей
12890,0,368124,61,начальное,3,вдовец / вдова,2,F,пенсионер,0,81273,покупка жилья для сдачи,"[покупка, , жилье, , для, , сдача, \n]",недвижимость,низкий (до 108т.р.),Нет детей


**Группировка по возрасту**

Также сгруппируем заемщиков по возрасту. Сможем использовать это для детализации в дальнейшем.

In [29]:
data['dob_years'].describe().round()

count    21454.0
mean        43.0
std         13.0
min          0.0
25%         33.0
50%         42.0
75%         53.0
max         75.0
Name: dob_years, dtype: float64

Разделим на четыре примерно равные по объему части, воспользовавшись для определения диапазонов информацией из *describe()*.

In [30]:
data['dob_years_grouped'] = pd.cut(data['dob_years'],[0, 33, 42, 53, np.inf], labels=["до 33 лет", "34-42 года", "43-53 года", "старше 53"])
data.sample(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas,purpose_main,total_income_cut,children_grouped,dob_years_grouped
2866,0,-809,46,среднее,1,женат / замужем,0,F,компаньон,0,297563,образование,"[образование, \n]",образование,высокий (более 196т.р.),Нет детей,43-53 года
9813,0,-2105,35,среднее,1,Не женат / не замужем,4,F,госслужащий,0,138143,покупка жилой недвижимости,"[покупка, , жилой, , недвижимость, \n]",недвижимость,ниже среднего (108-143т.р.),Нет детей,34-42 года
8407,2,-1371,0,среднее,1,женат / замужем,0,M,сотрудник,0,179138,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]",недвижимость,выше среднего (143-196т.р.),Два ребенка,NaN
16824,0,-498,58,среднее,1,гражданский брак,1,F,сотрудник,0,161079,свадьба,"[свадьба, \n]",свадьба,выше среднего (143-196т.р.),Нет детей,старше 53
8937,2,-1114,35,среднее,1,женат / замужем,0,F,сотрудник,0,127636,жилье,"[жилье, \n]",недвижимость,ниже среднего (108-143т.р.),Два ребенка,34-42 года


Сгруппируем на основании нового столбца.

In [31]:
data.groupby('dob_years_grouped')['debt'].agg(['count', 'mean']).reset_index().sort_values('mean')

,dob_years_grouped,count,mean
3,старше 53,5255,0.055947
2,43-53 года,5448,0.073238
1,34-42 года,5284,0.085731
0,до 33 лет,5366,0.109392


**Вывод**

На этапе предобработки удалили дубликаты, заполнили пропуски, выделили категории для более удобной группировки данных.  Удалось сохранить все строки за исключение дубликатов, заполнив пропуски с использованием медианных значений.
Данные готовы для финального этапа: анализа и ответана поставленные вопросы.

## Шаг 3. Анализируем закономерности

**- Есть ли зависимость между наличием детей и возвратом кредита в срок?**

Сгруппируем данные по количеству детей, используя **count** для подсчета количества заемщиков в каждой категории и **mean**, чтобы посчитать среднее. Т.к. в столбце **debt** значения из нулей и единиц, среднее значение будет являться долей просрочек по каждой категории. 

Отсортируем данные в таблице по столбцу в долей в порядк возрастания.

In [32]:
data.groupby('children_grouped')['debt'].agg(['count', 'mean']).reset_index().sort_values('mean')

,children_grouped,count,mean
0,Нет детей,14091,0.075438
3,Три и более,380,0.081579
1,Один ребенок,4855,0.091658
2,Два ребенка,2128,0.094925


**Вывод**

Заемщиков с тремя и более детьми в выборке не так много, поэтому эти данные не очень репрезентативны. В этой группе количество просрочек на среднем уровне.

Наибольшие проблемы с выплатами наблюдаются у заемщиков с детьми.
Самая большая группа - без детей - имеет наилучший показатель.
Можем сделать вывод, что заемщики без детей наиболее дисциплинировано расплачиваются по долгам. Возможно, это связано с тем, что они могут существенно урезать свои траты для погашения кредита в случае непредвиденных осложнений. Но причина может быть и в том, что заемщики без детей берут более мелкие суммы. Стоит рассмотреть этот показатель в связке с размером кредита и величиной дохода.  
Можем сделать вывод, что зависимость между наличием детей и просрочками существует: при наличии детей вероятность появления просрочки повышается.

**- Есть ли зависимость между семейным положением и возвратом кредита в срок?**

Проведем группировку по семейному положению.

In [33]:
data.groupby('family_status')['debt'].agg(['count', 'mean']).reset_index().sort_values('mean')

,family_status,count,mean
2,вдовец / вдова,959,0.065693
1,в разводе,1195,0.071130
4,женат / замужем,12339,0.075452
3,гражданский брак,4151,0.093471
0,Не женат / не замужем,2810,0.097509


Одинокие люди находятся в противоположных частях таблицы. Можно заметить, что наименьший процент просрочки среди вдовцов и находящихся в разводе. При этом люди, которые еще не вступали в брак, чаще всех допускают невыплаты. Вероятно, среди вдовцов и находящихся в разводе средний возраст выше, они более ответственны и обладают большим доходом, чем не женатые.

Также примечательно существенное отличие между женатыми заемщиками и заемщиками, проживающими в гражданском браке.  Доля просрочек заемщиков, проживающих в гражданском браке, существенно выше.


Построим сводную таблицу и взглянем на ситуацию с просрочками еще и в разрезе возрастных групп.

In [34]:
data_pivot = data.pivot_table(index=['family_status'], columns='dob_years_grouped', values='debt', aggfunc='mean')
display(data_pivot)

dob_years_grouped,до 33 лет,34-42 года,43-53 года,старше 53
family_status,,,,
Не женат / не замужем,0.119295,0.100182,0.089866,0.046512
в разводе,0.118644,0.063758,0.068602,0.057402
вдовец / вдова,0.076923,0.062500,0.058824,0.066865
гражданский брак,0.111309,0.100273,0.089423,0.066667
женат / замужем,0.103780,0.080891,0.066929,0.051582


**Вывод**

Наибольшие проблемы с просрочкой у молодых неженатых людей и/или живущих в гражданском браке. Среди людей старшего поколения количество просрочек существенно ниже.

**- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

Используем тот же метот для построения сводной таблицы по уровню дохода.

In [35]:
data.groupby('total_income_cut')['debt'].agg(['count', 'mean']).reset_index().sort_values('mean')

,total_income_cut,count,mean
3,высокий (более 196т.р.),5364,0.071402
0,низкий (до 108т.р.),5364,0.079605
2,выше среднего (143-196т.р.),5247,0.085382
1,ниже среднего (108-143т.р.),5479,0.088155


Попробуем дополнительно рассмотреть в разрезе возрастных групп. 

In [36]:
data_pivot = data.pivot_table(index=['total_income_cut'], columns='dob_years_grouped', values='debt', aggfunc='mean')
display(data_pivot)

dob_years_grouped,до 33 лет,34-42 года,43-53 года,старше 53
total_income_cut,,,,
низкий (до 108т.р.),0.111806,0.093085,0.074367,0.049880
ниже среднего (108-143т.р.),0.129291,0.098193,0.072646,0.055328
выше среднего (143-196т.р.),0.107168,0.087146,0.074405,0.067457
высокий (более 196т.р.),0.088617,0.068393,0.071764,0.054369


**Вывод**

Прямой зависимости между величиной дохода и надежностью заемщиков нет. Самый низкий процент наличия просрочек у наиболее обеспеченной группы. Заёмщики с низким доходом (до 108 000р) выплачивают кредит более дисциплинировано, чем заемщики со средним доходом. 
И снова наблюдаем, что среди самой младшей категории заемщиков наибольший процент неплатежей.

**- Как разные цели кредита влияют на его возврат в срок?**

Сгруппируем данные исходя из различных целей получения кредита. 

In [37]:
data.groupby('purpose_main')['debt'].agg(['count', 'mean']).reset_index().sort_values('mean')

,purpose_main,count,mean
1,недвижимость,10811,0.072334
3,свадьба,2324,0.080034
2,образование,4013,0.092200
0,автомобиль,4306,0.093590


**Вывод**

Хуже всего ситуация с кредитами на приобретение автомобиля и образование. Возможно люди не рассчитывают свои возможности при покупке авто, так же это могут быть спонтанные решения. 

Кредиты на образование - при сочетании обучения и работы, у большой части заемщиков может быть не высокий и/или не стабильный доход, и образование - как раз попытка решить эту проблему. Но на момент обучения возникают сложности с оплатой. 

Кредиты на свадьбу выплачиваются заметно лучше. Это может быть связано с тем, что это не большие суммы, и после свадьбы молодожены могут погасить часть долга за счет полученных в качестве подарка денежных средств.

Лучше всего ситуация с выплатами по кредитам, связанным с недвижимостью. Это самая большая категория - примерно половина от общего количества. Недвижимость - это крупная покупка, люди ответственнее и более взвешенно подходят к такому решению. Также по ипотечным кредитам чаще всего более низкий процент.


## Шаг 4. Общий вывод

Наиболее дисциплинированы заемщики старшего возраста, на их долю приходится минимальное количество просрочек. Причем наблюдается обратная корреляция между возрастом и долей просрочек: чем моложе, тем больше случаев неуплаты вовремя.
Заёмщики без детей чаще платят согласно установленному графику, а у заемщиков с детьми чаще возникают проблемы с оплатой.
Вдовцы и находящиеся в разводе платят стабильнее, чем неженатые.

Интересен факт, что высокий процент просрочек у заемщиков со средними доходами, а заемщики с низкими и высокими доходами погашают долги стабильнее.
Наибольшая доля просрочек по автокредитам, а самыми надежными являются кредиты связанные с недвижимостью. 


Для более глубоких выводов можно провести анализ, сгруппировав заемщиков по нескольким критериям. 

Полезными для анализа были бы данные о величине кредита и его сроке. Стоит рассмотреть возможность добавления этих данных в базу.